In [5]:
! pip install --quiet tqdm

In [2]:
import pandas as pd
import os
from lib.preprocess import preprocessor
from tqdm import tnrange, tqdm_notebook
import threading 

In [3]:
def process_readme(df):
    df = df[df.content.notna()].reset_index()
    df['content'] = df.content.map(preprocessor)
    df = df[df.content.notna()].reset_index()
    df['repo_owner'] = df.repo_name.map(lambda s: s.split('/')[0])
    return df

In [4]:
def process_claims(df):
    df['content'] = df.claims_text.map(preprocessor)
    df = df[df.claims_text.notna()].reset_index()
    return df

In [5]:
from  multiprocessing import Pool

In [ ]:
get_files = lambda inpath: list(os.walk(inpath))[0][2]

def preprocess(outfile, inpath, fn, id_key):
    pool = Pool()
    files = get_files(inpath)
    dfs = (pd.read_csv('{}/{}'.format(inpath, f)) for f in files)
    with open(outfile, 'w') as f:
        for df in tqdm_notebook(pool.map(fn, dfs)):
            for i,c in zip(df[id_key], df.content):
                f.write('{},{}\n'.format(i,c))
    pool.close()
    pool.join()

In [ ]:
preprocess('claims_lookup.csv',  'oss_bigquery_exports/patent_claims', 
                                                   process_claims, 
                                                   'application_number_formatted')

## Run for different datas

In [ ]:
preprocess('readme_lookup.csv', dfs, process_readmes, ('id', 'content'))

### Treat owner like collaborative filtering? 

In [101]:
by_owner = df.groupby('repo_owner').apply(lambda df: ' \t '.join(df.content))

In [102]:
with open('by_owner.txt', 'w') as f:
    for c in by_owner:
        f.write(c + '\n')